# Gradiant Descent Modeels - Data Preparation

In this notebook we focus on the data loading, basic exploring, and prepatation.

This notebook follows closely the previous data cleaning toturial from last week. We will be using the same dataset and producing the same output (not the output from the modified version you would have completed in your exercise).

In [3]:
pip install jupyter-summarytools

Note: you may need to restart the kernel to use updated packages.


## 1.0 Setup


In [ ]:
# import numpy and pandas libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
# set random seed to ensure that results are repeatable
np.random.seed(1)

## 2.0 Load data 

In [ ]:
# load data
airbnb = pd.read_csv("./data/airbnb.csv")

airbnb.head(3)

## 3.0 Conduct initial exploration of the data

We have a number of input variables and one target variable. For this analysis, the target variable is price.

First, our initial exploration of the data should answer the following questions:
1. How many rows and columns
2. How much of a problem do we have with na's?
3. What types of data are there?
4. What types of data are stored in columns
    1. identify which variables are numeric and may need to be standardized later
    2. identify which variables are categorical and may need to be transformed using and encoders such as one-hot-encoder.
5. Identify errors in the data - this is a common problem with categorical vars where the category is mispelled or spelled differently in some instances.
 

In [ ]:
# look at the data
airbnb.head(3) # note that we don't want to dump all the data to the screen

In [ ]:
# generate a basic summary of the data
airbnb.info()

In [ ]:
# generate a statistical summary of the numeric value in the data
airbnb.describe()

In [ ]:
# there are many ways we could explore our data. A rather new library available is called 
# jupyter-summarytools this library provides functions that provide very thorough summaries 
# of your data. Though such detail is not always required, there are times when you want a 
# thorough summary

# jupyter-summary tools is not part of the standard anaconda distribution of python, nor 
# is it in any conda channels. To install this library, you need to install it from the 
# terminal/command line using pip pip install jupyter-summarytools

# once installed, you can import this library and use dfSummary to provide a more thorough 
# summary of your data
import summarytools
from summarytools import dfSummary
dfSummary(airbnb)

In [ ]:
# Check the missing values by summing the total na's for each variable
airbnb.isna().sum()

In [ ]:
# create a list of these catagorical variables
category_var_list = list(airbnb.select_dtypes(include='object').columns)
category_var_list

In [ ]:
# explore the categorical variable values - often there are typos here that need to be fixed.
for cat in category_var_list: # generally, we want to avoid for loops and use a functional style (i.e. list comprehension)
    print(f"Category: {cat} Values: {airbnb[cat].unique()}")

### Summary the findings from our initial evaluation of the data

* We have 6 categorical variables
* We have 3 variables that have missing values
* There doesn't seem to be a problem with the catogorical class names.

## 4.0 Process the data

* Conduct any data prepartion that should be done *BEFORE* the data split.
* Split the data.
* Conduct any data preparation that should be done *AFTER* the data split.

### 4.1  Conduct any data prepartion that should be done *BEFORE* the data split

Tasks at this stage include:
1. Drop any columns/features 
2. Decide if you with to exclude any observations (rows) due to missing na's.
2. Conduct proper encoding of categorical variables
    1. You can transform them using dummy variable encoding, one-hot-encoding, or label encoding. 

#### Drop any columns/variables we will not be using

In [ ]:
# Our target is price; but there are three related price variableds - price, price_gte_150, 
# and price_category. We need to drop price_gte_150, and price_category
airbnb.drop(['price_category', 'price_gte_150'], axis=1, inplace = True)

#### Drop observations with too many NA's

If we want to remove the rows with NA's use the following code that is commented out. For this exercise - we will not drop rows with NA's 

In [ ]:
# If we want to remove rows with NA's use the following code:
# airbnb.dropna(axis=0, inplace=True)

In [ ]:
# verify that there are now no missing values
# airbnb.isna().sum()

In [ ]:
# investigage how many rows remain 
# airbnb.shape

#### Encode our categorical variables

Categorical variables usually have strings for their values. Many machine learning algorithms do not support string values for the input variables. Therefore, we need to replace these string values with numbers. This process is called categorical variable encoding.

In a previous step we identified 5 catagorical variables and found no indication of typos in the class names. Our focus is now on encoding the variables. 

We have three main approaches to encoding variables (these will be discussed in greater detail in class)
* One-Hot-Encoding
* Dummy Encoding
* Label Encoding

In this exercise; we will dummy encode neighbourhood_cleansed, property_type using dummy encoding, and room_type, bed_type and cancelation policy using label encoding. (we will have more discussion on these choices in class).

Before we do our encoding, we must identify if any of our categorical variables have a missing value. We will replace any missing values with the term 'unkown'.

In [ ]:
airbnb['property_type'].isna().sum() # check for missing values in this variable/column - we can see there are three for this variable

In [ ]:
airbnb["property_type"].fillna("unkown", inplace = True)

In [ ]:
airbnb['neighbourhood_cleansed'].isna().sum() 

In [ ]:
airbnb['room_type'].isna().sum() # can see by the results below, no missing values

In [ ]:
airbnb['bed_type'].isna().sum() # can see by the results below, no missing values

In [ ]:
airbnb['cancellation_policy'].isna().sum() # can see by the results below, no missing values

Now, let's encode neighborhood_cleansed and property_type as dummy variables and room_type, bed_type and cancelation_policy labeled (numeric)

In [ ]:
dummies_df = pd.get_dummies(airbnb['neighbourhood_cleansed'], prefix='neighbourhood_cleansed', drop_first=True)

In [ ]:
airbnb = airbnb.join(dummies_df)
airbnb.drop('neighbourhood_cleansed', axis=1, inplace = True)

In [ ]:
airbnb = airbnb.join(pd.get_dummies(airbnb['property_type'], prefix='property_type', drop_first=True))
airbnb.drop('property_type', axis=1, inplace = True)

In [ ]:
labelencoder = LabelEncoder()
airbnb['room_type'] = labelencoder.fit_transform(airbnb['room_type'])
airbnb['bed_type'] = labelencoder.fit_transform(airbnb['bed_type'])
airbnb['cancellation_policy'] = labelencoder.fit_transform(airbnb['cancellation_policy'])

In [ ]:
# explore the dataframe columns to verify encoding and dropped columns
airbnb.info()

### 4.2 Split data (train/test)

In [ ]:
# split the data into validation and training set
train_df, test_df = train_test_split(airbnb, test_size=0.3)

# to reduce repetition in later code, create variables to represent the columns
# that are our predictors and target
target = 'price'
predictors = list(airbnb.columns)
predictors.remove(target)

### 4.3  Conduct any data prepartion that should be done *AFTER* the data split

We will look at the following:
1) imput any missing numeric values using the mean of the variable/column
2) remove differences of scale by standardizing the numerica variables

#### Impute missing values

In [ ]:
numeric_cols_with_nas = list(train_df.isna().sum()[train_df.isna().sum() > 0].index)
numeric_cols_with_nas

We can see from the code above that there are 4 variables (columns) that contain missing numeric values (we've already taken care of any missing values in the catagorical variables earlier).

In [ ]:
imputer = SimpleImputer(strategy="median")

train_df[numeric_cols_with_nas] = imputer.fit_transform(train_df[numeric_cols_with_nas])
test_df[numeric_cols_with_nas] = imputer.transform(test_df[numeric_cols_with_nas])

#### Standardize numeric values

Now, let's create a common scale between the numberic columns by standardizing each numeric column

In [ ]:
# create a standard scaler and fit it to the training set of predictors
scaler = preprocessing.StandardScaler()
cols_to_stdize = ['latitude', 'longitude', 'accommodates', 
                   'bathrooms', 'bedrooms', 'beds', 'Number of amenities', 
                   'guests_included', 'price_per_extra_person', 'minimum_nights', 
                   'number_of_reviews', 'number_days_btw_first_last_review', 
                   'review_scores_rating']                
               
# Transform the predictors of training and validation sets
train_df[cols_to_stdize] = scaler.fit_transform(train_df[cols_to_stdize]) # train_predictors is not a numpy array


test_df[cols_to_stdize] = scaler.transform(test_df[cols_to_stdize]) # validation_target is now a series object


## 5.0 Save the data

In [ ]:
train_X = train_df[predictors]
train_y = train_df[target] # train_target is now a series objecttrain_df.to_csv('airbnb_train_df.csv', index=False)
test_X = test_df[predictors]
test_y = test_df[target] # validation_target is now a series object

train_df.to_csv('./data/airbnb_train_df_price.csv', index=False)
train_X.to_csv('./data/airbnb_train_X_price.csv', index=False)
train_y.to_csv('./data/airbnb_train_y_price.csv', index=False)
test_df.to_csv('./data/airbnb_test_df_price.csv', index=False)
test_X.to_csv('./data/airbnb_test_X_price.csv', index=False)
test_y.to_csv('./data/airbnb_test_y_price.csv', index=False)